In [49]:
with open(f'05.txt', 'r') as file:
    data = file.read().strip()

In [50]:
seeds = data.split('\n\n')[0].split(': ')[-1].split()
seeds = [int(seed) for seed in seeds]

maps = data.split('\n\n')[1:]
maps = [item.split(':\n')[1] for item in maps]
maps = [item.split('\n') for item in maps]
maps = [[item.split() for item in map] for map in maps]
maps = [[[int(number) for number in item] for item in map] for map in maps]

In [51]:
def get_destination(seed, map):
    for row in map:
        range_out_start, range_in_start, range_length = row
        if seed in range(range_in_start, range_in_start+range_length):
            return seed - range_in_start + range_out_start
    return seed

min_lowest_location = None

for number in seeds:
    latest = number
    for map in maps:
        latest = get_destination(latest, map)
    if not min_lowest_location or latest < min_lowest_location:
        min_lowest_location = latest

# puzzle 1
min_lowest_location

388071289

In [52]:
from itertools import batched

seeds_2 = [(item0, item0+item1) for item0, item1 in batched(seeds, 2)]

merged = []
for begin,end in sorted(seeds_2):
    if merged and merged[-1][1] >= begin - 1:
        merged[-1][1] = max(merged[-1][1], end)
    else:
        merged.append([begin, end])

In [57]:
merged

[[166086528, 210853524],
 [280549587, 984416942],
 [1054656969, 1223756736],
 [1233948799, 2045782636],
 [2326968141, 2396130363],
 [2600461189, 2692794035],
 [2698492851, 2713095920],
 [2755327667, 3104327198],
 [3169137700, 3440855309],
 [3522125441, 3545501536]]

In [56]:
min_lowest_location = None

for item0, item1 in merged:
    for number in range(item0, item1):
        latest = number
        for map in maps:
            latest = get_destination(latest, map)
        if not min_lowest_location or latest < min_lowest_location:
            min_lowest_location = latest

# puzzle 2
min_lowest_location

166086528


KeyboardInterrupt: 

In [55]:
len(merged)

10